![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [111]:
# Import packages
import pandas as pd
import numpy as np

In [112]:
# Load and read the dataset
df = pd.read_csv("bank_marketing.csv")
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [113]:
# Print value counts for specific columns
for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [114]:
# Check column names
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [115]:
# List columns for each new file
client_cols = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']
campaign_cols = ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']
economics_cols = ['client_id', 'cons_price_idx', 'euribor_three_months']

# Subset data for new DataFrames
client = df[client_cols]
campaign = df[campaign_cols]
economics = df[economics_cols]

In [116]:
# Check client types
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default    object
mortgage          object
dtype: object

In [117]:
# Checking values for client columns
for col in ['job', 'education', 'credit_default', 'mortgage']:
    print(col, '\n', '----------\n', client[col].value_counts(), '\n')

job 
 ----------
 admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64 

education 
 ----------
 university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64 

credit_default 
 ----------
 no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64 

mortgage 
 ----------
 yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64 



In [118]:
# Cleaning data on client columns and confirming changes
client['job'] = client['job'].str.replace('.', '_').str.strip()
client['education'] = client['education'].str.replace('.', '_').replace('unknown', np.nan).str.strip()
client['credit_default'] = np.where(client['credit_default'].str.strip() == 'yes', 1, 0).astype(bool)
client['mortgage'] = np.where(client['mortgage'].str.strip() == 'yes', 1, 0).astype(bool)

print(client.dtypes)
client.head(20)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
5,5,45,services,married,basic_9y,False,False
6,6,59,admin_,married,professional_course,False,False
7,7,41,blue-collar,married,NaN,False,False
8,8,24,technician,single,professional_course,False,True
9,9,25,services,single,high_school,False,True


In [119]:
# Checking values on campaign columns
for col in ['previous_outcome', 'campaign_outcome']:
    print(col, '\n', '----------\n', campaign[col].value_counts(), '\n')

previous_outcome 
 ----------
 nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64 

campaign_outcome 
 ----------
 no     36548
yes     4640
Name: campaign_outcome, dtype: int64 



In [120]:
# Changing boolean columns
campaign['previous_outcome'] = np.where(campaign['previous_outcome'].str.strip() == 'success', 1, 0).astype(bool)
campaign['campaign_outcome'] = np.where(campaign['campaign_outcome'].str.strip() == 'yes', 1, 0).astype(bool)

# Creating month dictionary
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
dict_months = dict(zip(months, list(range(1,13))))

# Replacing month values, adding year column and creating contact date column
df['month'] = df['month'].replace(dict_months.keys(), dict_months.values())
df['year'] = 2022
campaign['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']], format='%Y-%m-%d')

campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [121]:
# Saving files as CSVs
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)